In [1]:
import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from typing import Optional, Union, List
from urllib.parse import urljoin

class TJPRScraper:
    """Scraper for the Court of Justice of Paraná com download de PDFs."""

    BASE_URL = "https://portal.tjpr.jus.br/jurisprudencia/publico/pesquisa.do?actionType=pesquisar"
    HOME_URL = "https://portal.tjpr.jus.br/jurisprudencia/"
    USER_AGENT = (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0"
    )

    def __init__(self, download_dir: str = "pdfs"):
        self.session = requests.Session()
        self.token: Optional[str] = None
        self.jsessionid: Optional[str] = None
        self.download_dir = download_dir
        os.makedirs(download_dir, exist_ok=True)

        # Configurar headers padrão
        self.session.headers.update({
            'User-Agent': self.USER_AGENT,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Connection': 'keep-alive',
        })

    def _get_initial_tokens(self):
        """
        Extracts the JSESSIONID and the token from the TJPR initial page.
        """
        resp = self.session.get(self.HOME_URL)
        resp.raise_for_status()
        self.jsessionid = self.session.cookies.get('JSESSIONID')
        token = None
        soup = BeautifulSoup(resp.text, "html.parser")
        for a in soup.find_all('a', href=True):
            m = re.search(r'tjpr\.url\.crypto=([a-f0-9]+)', a['href'])
            if m:
                token = m.group(1)
                break
        if not token:
            raise RuntimeError("Não foi possível extrair o token da página inicial.")
        self.token = token
        return self.jsessionid, token

    def _get_ementa_completa(self, id_processo, criterio):
        """
        Fetches the complete minute of a process from TJPR.
        """
        url = (
            "https://portal.tjpr.jus.br/jurisprudencia/publico/pesquisa.do?"
            "actionType=exibirTextoCompleto"
            f"&idProcesso={id_processo}&criterio={criterio}"
        )
        headers = {
            'accept': 'text/javascript, text/html, application/xml, text/xml, */*',
            'accept-language': 'pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'cache-control': 'no-cache',
            'pragma': 'no-cache',
            'referer': (
                'https://portal.tjpr.jus.br/jurisprudencia/publico/pesquisa.do?actionType=pesquisar'
            ),
            'user-agent': self.USER_AGENT,
            'x-prototype-version': '1.5.1.1',
            'x-requested-with': 'XMLHttpRequest',
        }
        cookies = {'JSESSIONID': self.jsessionid}
        resp = self.session.get(url, headers=headers, cookies=cookies)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, 'html.parser').get_text("\n", strip=True)

    def cjsg_download(self, termo: str, paginas: Union[int, list, range] = 1,
                     data_julgamento_de: str = None, data_julgamento_ate: str = None,
                     data_publicacao_de: str = None, data_publicacao_ate: str = None) -> list:
        """
        Downloads raw results from the TJPR jurisprudence search (multiple pages).
        Returns a list of HTMLs (one per page).
        """
        if not self.jsessionid:
            self._get_initial_tokens()

        url = "https://portal.tjpr.jus.br/jurisprudencia/publico/pesquisa.do?actionType=pesquisar"
        headers = {
            'accept-language': 'pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'cache-control': 'no-cache',
            'content-type': 'application/x-www-form-urlencoded',
            'origin': 'https://portal.tjpr.jus.br',
            'pragma': 'no-cache',
            'referer': url,
            'user-agent': self.USER_AGENT,
        }
        cookies = {'JSESSIONID': self.jsessionid}

        if isinstance(paginas, int):
            paginas_iter = range(1, paginas+1)
        else:
            paginas_iter = list(paginas)

        resultados = []
        for pagina_atual in tqdm(paginas_iter, desc='Baixando páginas TJPR'):
            data = {
                'usuarioCienteSegredoJustica': 'false',
                'segredoJustica': 'pesquisar com',
                'id': '',
                'chave': '',
                'dataJulgamentoInicio': data_julgamento_de or '',
                'dataJulgamentoFim': data_julgamento_ate or '',
                'dataPublicacaoInicio': data_publicacao_de or '',
                'dataPublicacaoFim': data_publicacao_ate or '',
                'processo': '',
                'acordao': '',
                'idComarca': '',
                'idRelator': '',
                'idOrgaoJulgador': '',
                'idClasseProcessual': '',
                'idAssunto': '',
                'pageVoltar': pagina_atual - 1,
                'idLocalPesquisa': '1',
                'ambito': '-1',
                'descricaoAssunto': '',
                'descricaoClasseProcessual': '',
                'nomeComarca': '',
                'nomeOrgaoJulgador': '',
                'nomeRelator': '',
                'idTipoDecisaoAcordao': '',
                'idTipoDecisaoMonocratica': '',
                'idTipoDecisaoDuvidaCompetencia': '',
                'criterioPesquisa': termo,
                'pesquisaLivre': '',
                'pageSize': 10,
                'pageNumber': pagina_atual,
                'sortColumn': 'processo_sDataJulgamento',
                'sortOrder': 'DESC',
                'page': pagina_atual - 1,
                'iniciar': 'Pesquisar',
            }
            resp = self.session.post(url, data=data, headers=headers, cookies=cookies)
            resp.raise_for_status()
            resultados.append(resp.text)
        return resultados

    def _extract_acordao_links(self, html_content: str) -> list:
        """
        Extrai todos os links para páginas de acórdãos dos resultados
        """
        soup = BeautifulSoup(html_content, "html.parser")
        links = []

        # Encontrar a tabela de resultados
        tabela = soup.find('table', class_='resultTable jurisprudencia')
        if not tabela:
            print("Tabela de resultados não encontrada!")
            return links

        # Encontrar todas as linhas da tabela (ignorando o cabeçalho)
        for row in tabela.find_all('tr')[1:]:
            # Encontrar a primeira célula da linha
            primeira_celula = row.find('td')
            if not primeira_celula:
                continue

            # Encontrar o link do acórdão dentro da primeira célula
            link_tag = primeira_celula.find('a', class_='acordao negrito')
            if link_tag and link_tag.has_attr('href'):
                full_url = urljoin(self.HOME_URL, link_tag['href'])
                links.append(full_url)

        return links

    def _extract_pdf_url(self, acordao_url: str) -> Optional[str]:
        """
        Extrai o link do PDF de uma página de acórdão, tratando links JavaScript
        """
        try:
            resp = self.session.get(acordao_url)
            resp.raise_for_status()

            soup = BeautifulSoup(resp.text, 'html.parser')

            # Encontrar o link do PDF usando o ícone iPdf.gif
            pdf_img = soup.find('img', src='/jurisprudencia/img/iPdf.gif')
            if pdf_img and pdf_img.parent and pdf_img.parent.name == 'a':
                pdf_link = pdf_img.parent
                href = pdf_link.get('href', '')

                # Se for um link JavaScript, extraímos o URL real
                if href.startswith("javascript:document.location.replace("):
                    match = re.search(r"replace\('([^']+)'\)", href)
                    if match:
                        relative_url = match.group(1)
                        return urljoin(self.HOME_URL, relative_url)
                else:
                    return urljoin(self.HOME_URL, href)

        except Exception as e:
            print(f"Erro ao acessar {acordao_url}: {str(e)}")

        return None

    def _download_pdf(self, pdf_url: str, processo: str):
        """
        Baixa um arquivo PDF e salva com nome baseado no processo
        """
        try:
            # Sanitiza o número do processo para usar como nome de arquivo
            safe_processo = re.sub(r'[^a-zA-Z0-9]', '_', processo)[:100]
            filename = f"{safe_processo}.pdf"
            filepath = os.path.join(self.download_dir, filename)

            # Configurar headers para a requisição do PDF
            headers = {
                'User-Agent': self.USER_AGENT,
                'Referer': self.BASE_URL,
                'Accept': 'application/pdf, */*'
            }

            response = self.session.get(pdf_url, headers=headers, stream=True)
            response.raise_for_status()

            # Verificar se o conteúdo é um PDF válido
            content_type = response.headers.get('Content-Type', '')
            if 'pdf' not in content_type.lower():
                # Verificar a assinatura do PDF
                if response.content[:4] != b'%PDF':
                    print(f"AVISO: O conteúdo não parece ser um PDF válido: {pdf_url}")
                    return False

            # Salvar o arquivo
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            # Verificar tamanho do arquivo
            if os.path.getsize(filepath) < 1024:  # Menos de 1KB
                print(f"AVISO: PDF muito pequeno, possivelmente inválido: {filepath}")
                return False

            return True
        except Exception as e:
            print(f"Erro ao baixar {pdf_url}: {str(e)}")
            return False

    def cjsg_parse(self, resultados_brutos: list, criterio: str = None) -> pd.DataFrame:
        """
        Extracts relevant data from the HTMLs returned by TJPR.
        Returns a DataFrame with the decisions.
        """
        resultados = []
        for html in resultados_brutos:
            soup = BeautifulSoup(html, "html.parser")
            tabela = soup.select_one("table.resultTable.jurisprudencia")
            if not tabela:
                continue
            linhas = tabela.find_all("tr")[1:]  # pula o cabeçalho
            for row in linhas:
                cols = row.find_all("td")
                if len(cols) < 2:
                    continue
                dados_td = cols[0]
                ementa_td = cols[1]
                # Processo
                processo = ''
                processo_a = dados_td.find('a', class_='decisao negrito')
                if processo_a:
                    processo = processo_a.get_text(strip=True)
                else:
                    for div in dados_td.find_all('div'):
                        if 'Processo:' in div.get_text():
                            processo_div = div.find_all('div')
                            if processo_div:
                                processo = processo_div[0].get_text(strip=True)
                # Relator
                relator = ''
                relator_label = dados_td.find(string=lambda t: t and 'Relator:' in t)
                if relator_label:
                    relator = relator_label.split('Relator:')[-1].strip()
                    if not relator:
                        next_sib = relator_label.parent.find_next_sibling(text=True)
                        if next_sib:
                            relator = next_sib.strip()
                # Órgão julgador
                orgao_julgador = ''
                orgao_label = dados_td.find(string=lambda t: t and 'Órgão Julgador:' in t)
                if orgao_label:
                    orgao_julgador = orgao_label.split('Órgão Julgador:')[-1].strip()
                # Data julgamento
                data_julgamento = ''
                data_label = dados_td.find(string=lambda t: t and 'Data Julgamento:' in t)
                if data_label:
                    data_julgamento = data_label.split('Data Julgamento:')[-1].strip()
                    if not data_julgamento:
                        next_sib = data_label.parent.find_next_sibling(text=True)
                        if next_sib:
                            data_julgamento = next_sib.strip()
                # Ementa
                ementa = ementa_td.get_text("\n", strip=True)
                # Detecta "Leia mais..." e busca a ementa completa
                if 'leia mais' in ementa.lower():
                    input_id = dados_td.find('input', {'name': 'idsSelecionados'})
                    if input_id and 'value' in input_id.attrs:
                        id_processo = input_id['value']
                    else:
                        id_processo = ''
                    if id_processo and criterio and self.session and self.jsessionid:
                        try:
                            ementa = self._get_ementa_completa(id_processo, criterio)
                        except (requests.RequestException, AttributeError) as e:
                            ementa += (f"\n[Erro ao buscar ementa completa: {e}]")
                resultados.append({
                    'processo': processo,
                    'orgao_julgador': orgao_julgador,
                    'relator': relator,
                    'data_julgamento': data_julgamento,
                    'ementa': ementa,
                })
        df = pd.DataFrame(resultados)
        if not df.empty and 'data_julgamento' in df.columns:
            df['data_julgamento'] = pd.to_datetime(
                df['data_julgamento'], errors='coerce', dayfirst=True
            ).dt.date
        return df

    def download_acordaos_pdf(self, resultados_brutos: list):
        """
        Processa os resultados brutos para extrair e baixar os PDFs dos acórdãos
        """
        if not resultados_brutos:
            print("Nenhum resultado bruto para processar.")
            return

        if not self.jsessionid:
            self._get_initial_tokens()

        for i, html in enumerate(resultados_brutos):
            print(f"Processando página {i+1}/{len(resultados_brutos)}")

            # Extrai links para as páginas dos acórdãos
            acordao_links = self._extract_acordao_links(html)

            if not acordao_links:
                print(f"  Nenhum acórdão encontrado na página {i+1}")
                continue

            print(f"  Encontrados {len(acordao_links)} acórdãos")

            for j, link in enumerate(acordao_links):
                print(f"    Processando acórdão {j+1}/{len(acordao_links)}")

                # Extrai o link do PDF da página do acórdão
                pdf_url = self._extract_pdf_url(link)

                if pdf_url:
                    # Extrai o número do processo da URL
                    processo_match = re.search(r'Acórdão-([\w.-]+)', link)
                    processo = processo_match.group(1) if processo_match else f"acordao_{i+1}_{j+1}"

                    # Faz o download do PDF
                    if self._download_pdf(pdf_url, processo):
                        print(f"      Download realizado: {processo}.pdf")
                    else:
                        print(f"      Falha no download: {pdf_url}")
                else:
                    print(f"      Link PDF não encontrado para: {link}")

    def cjsg(self, termo: str, paginas: Union[int, list, range] = 1,
             data_julgamento_de: str = None, data_julgamento_ate: str = None,
             data_publicacao_de: str = None, data_publicacao_ate: str = None, **kwargs) -> pd.DataFrame:
        """
        Busca jurisprudência e baixa os PDFs automaticamente
        """
        brutos = self.cjsg_download(
            termo=termo,
            paginas=paginas,
            data_julgamento_de=data_julgamento_de,
            data_julgamento_ate=data_julgamento_ate,
            data_publicacao_de=data_publicacao_de,
            data_publicacao_ate=data_publicacao_ate,
            **kwargs
        )

        # Processa e baixa os PDFs
        self.download_acordaos_pdf(brutos)

        # Retorna o DataFrame com os metadados
        return self.cjsg_parse(brutos, termo)

    def cpopg(self, id_cnj: Union[str, List[str]]):
        """Stub: Primeiro grau case consultation not implemented for TJPR."""
        raise NotImplementedError("Consulta de processos de 1º grau não implementada para TJPR.")

    def cposg(self, id_cnj: Union[str, List[str]]):
        """Stub: Segundo grau case consultation not implemented for TJPR."""
        raise NotImplementedError("Consulta de processos de 2º grau não implementada para TJPR.")

In [2]:
# Instanciar o scraper
scraper = TJPRScraper(download_dir="pdfs_tjpr")

# Executar a pesquisa com termo e datas
df = scraper.cjsg(
    termo="IDPJ",
    paginas=5,
    data_julgamento_de="01/01/2025",
    data_julgamento_ate="31/12/2025"
)

# Salvar os resultados em CSV
df.to_csv("jurisprudencia_tjpr.csv", index=False, encoding="utf-8-sig")

# Os PDFs estarão na pasta especificada

Baixando páginas TJPR:   0%|          | 0/5 [00:00<?, ?it/s]

Processando página 1/5
  Encontrados 8 acórdãos
    Processando acórdão 1/8
      Download realizado: 0126386-20.2024.8.16.0000.pdf
    Processando acórdão 2/8
      Download realizado: 0122818-93.2024.8.16.0000.pdf
    Processando acórdão 3/8
      Download realizado: 0019928-42.2025.8.16.0000.pdf
    Processando acórdão 4/8
      Download realizado: 0123090-87.2024.8.16.0000.pdf
    Processando acórdão 5/8
      Download realizado: 0122816-26.2024.8.16.0000.pdf
    Processando acórdão 6/8
      Download realizado: 0133894-17.2024.8.16.0000.pdf
    Processando acórdão 7/8
      Download realizado: 0022871-32.2025.8.16.0000.pdf
    Processando acórdão 8/8
      Download realizado: 0019762-10.2025.8.16.0000.pdf
Processando página 2/5
  Encontrados 9 acórdãos
    Processando acórdão 1/9
      Download realizado: 0027936-08.2025.8.16.0000.pdf
    Processando acórdão 2/9
      Download realizado: 0097401-41.2024.8.16.0000.pdf
    Processando acórdão 3/9
      Download realizado: 0010550-62